# Summary
In this notebook we will setup a *TSEB* model to run a time series data at point scale.

# TSEB setup
The main input data consists on an ASCII table with tab separators and the firs row containing the name of the variables. At least the table should contain the following field names (case sensitive):
>* **TSEB-PT** : `Year, DOY, Time, Trad, VZA, Ta, u, ea, Sdn, LAI & hc`
>* **DTD** : `Year, DOY, Time, Trad_0, Trad_1, VZA, Ta_0, Ta_1, u, ea, Sdn, LAI & hc`
>* **TSEB-2T** : `Year, DOY, Time, Tc, Ts, Ta, u, ea, Sdn, LAI & hc`

**Year** is not really used in the model but just kept in case several years were processed, **DOY** is the day of the year (1-366), **Time** is the measurement time in decimal hours (0-24). **Trad** is the radiometric composite temperature; **Trad_0** and **Trad_1**, the same for time 0 (near sunrise0 and time 1 (around noon) for *DTD* model: **Ts** and **Tc** are the component soil and canopy temperatures for *TSEB-2T*; **Ta** is the air temperature: all temperatures expressed in Celsius. **u** is the wind speed measurement (m s$^{-1}$), **ea** is the atmospheric vapour pressure (mb), **Sdn** is the shortwave irradiance (W m$^{-2}$), **LAI** is the effective leaf area index, and **hc** is the canopy height (m).

Additional input variables that can be included in the table are: **SAA** solar azimuth angle, **SZA** solar zenith angle, **Ldn** incoming longwave radiation (W m$^{-2}$), **f_C** fractional cover, **f_g** green fraction, **wc** canopy width to height ratio, or **G'** soil heat flux (W m$^{-2}$)

If any of those additional variables are not found in the table they will be internally estimated by TSEB or use default values. The order of the columns is not relevant, and neither whethere there are additional columns in the table (they will be ignored if their names do not match any of the possible input variables.

## Select the input and output filles and then fill all the information in each tab:
You can press the Load Configuration File to load a configuration text file which will upload all its information in the corresponding cell.

You can also press the Save Configuration File button to save the configuration in a text file which could be used in further runs

In [ ]:
from pyTSEB import PyTSEB # import the PyTSEB class object in the pyTSEB.py module
setup=PyTSEB() # Create the setup instance from the PyTSEB class object
setup.PointTimeSeriesWidget() # Load the TSEB configuration Widget

# Run TSEB
Once TSEB is configured we will parse all the information in the widgets to run TSEB. A progress bar will show up and once done it will write an ASCII table in the output txt file with the following variables:
>Year, DOY, Time, LAI, f_g, skyl, VZA, SZA, SAA, Ldn, Rn_model, Rn_sw_veg, Rn_sw_soil, Rn_lw_veg, Rn_lw_soil, Tc, Ts, Tac, LE_model, H_model, LE_c, H_c, LE_s, H_s, flag, zo, d, G_model, R_s, R_x, R_a, u_friction, L, n_iterations

where *f_g* is the green fraction, *skyl* is the ratio of difuse radiation, *VZA*, *SZA* and *SAA* are view and solar angles, *Ldn* is downwelling longwave radiation, *Rn* is net radiation (*sw* and *lw* for shortwave and longwave, *veg* and *soil* for canopy and soil), *Tc* and *Ts* are canopy and soil temperatures, *LE* is latent heat flux , *H* is sensible heat flux, *flag* is a quality flag (255==BAD), *zo* and *d* are roughness leght and zero-plane displacement height, *G* is soil heat flux, *R_s*, *R_x* and *R_a* are resistances to heat and momentum transport, *u_friction* is friction velocity, *L* is the Monin-Obukhov lenght and *n_iterations* is the number of iterations in TSEB to achieve converge stability.

In [ ]:
# Get the data from the widgets
setup.GetDataTSEBWidgets(False)
#run TSEB
setup.RunTSEBPointSeries()

# Display results
Now we can open the image and display the TSEB outputs

In [ ]:
import numpy as np
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
output_notebook()
# get the data
output=np.genfromtxt(setup.OutputFile,names=True,dtype=None)
# create a datetime variable
timestamp=output['Year']*1000+output['DOY']+output['Time']/24
    
p = figure(title="Time Series", plot_width=600, plot_height=400)
# Plot all the points as dashed lines
p.line(x=timestamp,y=output['H_model'], color='red',line_dash=[4,4])
p.line(x=timestamp,y=output['LE_model'], color='blue', line_dash=[4,4])
p.line(x=timestamp,y=output['Rn_model'], color='black', line_dash=[4,4])
p.line(x=timestamp,y=output['G_model'], color='green', line_dash=[4,4])

# Filter only good quatlity data points flag<2:
H=np.array(output['H_model'])
H[output['flag']>=2]=float('nan')
LE=np.array(output['LE_model'])
LE[output['flag']>=2]=float('nan')
Rn=np.array(output['Rn_model'])
Rn[output['flag']>=2]=float('nan')
G=np.array(output['G_model'])
G[output['flag']>=2]=float('nan')
#Plot good quality data as straight lines
p.line(x=timestamp,y=H, color='red', legend='H')
p.line(x=timestamp,y=LE, color='blue', legend='LE')
p.line(x=timestamp,y=Rn, color='black', legend='Rn')
p.line(x=timestamp,y=G, color='green', legend='G')

show(p);

* [Click here for know more about the TSEB model](./TSEB_and_Resistances.ipynb "The TSEB model")
* [To know more about the TSEB code press here](./pyTSEB_in_Detail.ipynb "Introduction to TSEB")